In [1]:
from amacrine_color_schema import *

Connecting sstrauss@172.25.240.205:3306


Load kernels

In [2]:
key_kernel = {
                'experimenter': 'Korympidou',
                'artefact_kernel_id': 1,
                'preprocess_param_set_id': 1,
                'stim_id': 15,
                'noise_id': 1,
                'param_id': 1
            }

kernels = pd.DataFrame.from_dict(((CorrectedBGFlickerKernels() - 
                                   [Pharmacology() & 'control_flag = 0' & 'treatment != "control"'] - 
                                   [Indicator() & 'virustransprotein = "GCaMP8f"']) *
                                  BGFullResponses() *
                                  IplDepth() *
                                  RelativeFieldLocation() *
                                  QualityLocation() *
                                  MinMaxQuality() &
                                  key_kernel).fetch(as_dict=True))

Unravel dataframe so that each row is one ROI

In [3]:
columns = ['date', 'exp_num', 'field_id', 'roi', 
           'uv_center', 'uv_ring', 'uv_surround', 
           'green_center', 'green_ring', 'green_surround', 
           'ipl_depth', 'quality_values', 
           'Rel. field location y', 'quality_location',
           'response_trace_flicker']
df = pd.DataFrame(columns=columns)

for index, row in kernels.iterrows():
    
    nb_rois = row['st_kernels_uv_corrected'].shape[1]

    current_data = {'date': np.repeat(row['date'], nb_rois), 
                    'exp_num': np.repeat(row['exp_num'], nb_rois), 
                    'field_id': np.repeat(row['field_id'], nb_rois), 
                    'roi': np.arange(1,nb_rois+1),
                    'uv_center': list(row['st_kernels_uv_corrected'][:,:,0].T),
                    'uv_ring': list(row['st_kernels_uv_corrected'][:,:,1].T), 
                    'uv_surround':  list(row['st_kernels_uv_corrected'][:,:,2].T),
                    'green_center':  list(row['st_kernels_green_corrected'][:,:,0].T), 
                    'green_ring':  list(row['st_kernels_green_corrected'][:,:,1].T), 
                    'green_surround':  list(row['st_kernels_green_corrected'][:,:,2].T),
                    'ipl_depth': row['depth'], 
                    'quality_values': row['quality_values'],
                    'Rel. field location y': np.repeat(row['rely'], nb_rois),
                    'quality_location': np.repeat(row['quality_location'], nb_rois),
                    'response_trace_flicker': list(row['full_response_traces'])
                   }
    
    df = pd.concat([df, pd.DataFrame(current_data, columns = columns)], ignore_index = True)

Quality of optic disk recording

In [4]:
new_column = np.repeat('aaaaaaaaaa', len(df))
loc = df['Rel. field location y'].to_numpy()
new_column[np.where(loc < 0)[0]] = 'ventral'
new_column[np.where(loc > 0)[0]] = 'dorsal'
print(np.unique(new_column))
quality_rois = df['quality_location'].to_numpy()
new_column[np.where(quality_rois == 0)[0]] = 'no_info'
print(np.unique(new_column))
df['y_retinal_location'] = new_column

assert np.all(df['y_retinal_location'][df['quality_location'] == 0] == 'no_info')
assert np.all(df['y_retinal_location'][df['quality_location'] != 0] != 'no_info')

['dorsal' 'ventral']
['dorsal' 'no_info' 'ventral']


Filter ROIs by ipl depth

In [5]:
ipl_rois = df[(df['ipl_depth'] < 1) & (df['ipl_depth'] > 0)]
ipl_rois = ipl_rois.reset_index(drop=True)

print(np.amax(ipl_rois['ipl_depth']))
print(np.amin(ipl_rois['ipl_depth']))

0.9998798
9.918213e-05


Quality filter ROIs

In [6]:
final_df = ipl_rois[ipl_rois['quality_values']>0.1]
final_df = final_df.reset_index(drop=True)

print(np.amin(final_df['quality_values']))

0.10001667453491629


Load Chirp and Local Chirp

In [7]:
key_chirp_g = {
                'experimenter': 'Korympidou',
                'stim_id': 5,
                'preprocess_param_set_id': 1,
                'resample_f_id': 3
            }
global_chirp = pd.DataFrame.from_dict(((NormalizedChirp() - 
                                        [Pharmacology() & 'control_flag = 0' & 'treatment != "control"'] - 
                                        [Indicator() & 'virustransprotein = "GCaMP8f"']) & 
                                       key_chirp_g).fetch(as_dict=True))
global_chirp = global_chirp.rename(columns={'time_bin': 'time_bin_global_chirp',
                                            'normalized_chirp': 'global_chirp'})


key_chirp_l = {
                'experimenter': 'Korympidou',
                'stim_id': 6,
                'preprocess_param_set_id': 1,
                'resample_f_id': 3
                }
local_chirp = pd.DataFrame.from_dict(((NormalizedChirp() - 
                                       [Pharmacology() & 'control_flag = 0' & 'treatment != "control"'] - 
                                       [Indicator() & 'virustransprotein = "GCaMP8f"']) & 
                                      key_chirp_l).fetch(as_dict=True))
local_chirp = local_chirp.rename(columns={'time_bin': 'time_bin_local_chirp', 
                                          'normalized_chirp': 'local_chirp'})

Drop unnecessary columns

In [8]:
drop = ['experimenter', 'stim_id', 'presentation_id', 'preprocess_param_set_id', 'resample_f_id']

global_chirp = global_chirp.drop(drop, axis = 1)
local_chirp = local_chirp.drop(drop, axis = 1)

Merge different dataframes

In [9]:
match_on = ['date', 'exp_num', 'field_id', 'roi']

interm = final_df.merge(global_chirp, how='inner', on=match_on)
all_data = interm.merge(local_chirp, how='inner', on=match_on)

Check time bins

In [10]:
referenceA = all_data['time_bin_local_chirp'].iloc[0]
referenceB = all_data['time_bin_global_chirp'].iloc[0]
assert np.all(referenceA == referenceB)

for index, row in all_data.iterrows():
    assert np.all(referenceA == row['time_bin_local_chirp'])
    assert np.all(referenceB == row['time_bin_global_chirp'])

Add batch, mouse, eye to the dataframe

In [11]:
grouped_df = all_data.groupby(['date', 'exp_num', 'field_id'])
field_ID = -999*np.ones(len(all_data))
current_batch = 0
for name, group in grouped_df:
    group_indices = group.index.to_numpy()
    field_ID[group_indices] = current_batch
    current_batch = current_batch + 1
all_data['field'] = field_ID

grouped_df = all_data.groupby(['date', 'exp_num'])
eye_ID = -999*np.ones(len(all_data))
current_eye = 0
for name, group in grouped_df:
    group_indices = group.index.to_numpy()
    eye_ID[group_indices] = current_eye
    current_eye = current_eye + 1
all_data['eye'] = eye_ID

grouped_df = all_data.groupby(['date'])
mouse_ID = -999*np.ones(len(all_data))
current_mouse = 0
for name, group in grouped_df:
    group_indices = group.index.to_numpy()
    mouse_ID[group_indices] = current_mouse
    current_mouse = current_mouse + 1
all_data['mouse'] = mouse_ID

In [12]:
print(np.unique(all_data['field']).shape)
print(np.unique(all_data['date']))

(79,)
[datetime.date(2021, 6, 29) datetime.date(2021, 7, 7)
 datetime.date(2021, 7, 9) datetime.date(2021, 10, 19)
 datetime.date(2021, 10, 21) datetime.date(2022, 1, 21)
 datetime.date(2022, 1, 28) datetime.date(2022, 1, 31)
 datetime.date(2022, 2, 2) datetime.date(2022, 4, 8)
 datetime.date(2022, 4, 13) datetime.date(2022, 4, 14)
 datetime.date(2022, 5, 9) datetime.date(2022, 5, 11)
 datetime.date(2022, 5, 13) datetime.date(2022, 5, 18)
 datetime.date(2022, 5, 19)]


Save dataframe

In [13]:
#all_data.to_pickle('Data/ACs/Dataframes/control_data_full.pkl')